In [ ]:
import pickle
import numpy as np
import PIL.Image
import itertools
from IPython.core.display import Image, display
import scipy.ndimage
import random
import numpy as np
import matplotlib.pylab as plt
import tqdm
import math
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import figure
%matplotlib inline

# Parameters
random_seed = 42
random_std_dev = 1e-4
norm_epsilon = 1e-4
epochs = 5000
eta = 4.2e-6
acc_eps = 5e-3
lamda = 1e-4

# Functions
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def model(x, w):
    return sigmoid(np.dot(w.T, x))

def logloss(x, y, w):
    return sum(np.log(1. + np.exp(-y[i] * np.dot(w.T, x[i]))) for i in range(len(y))) / len(y)

def accuracy(pred, true):
    return sum(float(t - p <= acc_eps) if t == 1 else float(p <= acc_eps) for p, t in zip(pred, true)) / len(true)

def gradient_loss(x, y, w, j):
    return - y[j] * x[j] / (1 + np.exp(y[j] * np.dot(w.T, x[j]))) + lamda * w

def evaluation(pred, x, y, w):
    return [logloss(x, y, w), accuracy(pred, y)]

def data_normalization(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data - mean) / (std + norm_epsilon)
    
with open('./hw_1_train.pickle', 'rb') as f:
    train = pickle.load(f)

with open('./hw_1_test_no_lables.pickle', 'rb') as f:
    test = pickle.load(f)

data = train['data']
labels = train['labels']

# Normalization of the input data
data = data_normalization(data)

# Tag Conversion
labels[labels==5] = -1
labels[labels==6] = 1

# Splitting data on the training and the test samples
x_train, x_test, y_train, y_test = train_test_split(data, labels, random_state=10)

# Arrays' dimensions
width = x_train.shape[1]
tr_length = x_train.shape[0]
te_length = x_test.shape[0]

# Random weight initialization
np.random.seed(random_seed)
w = np.random.normal(scale=random_std_dev, size=width)

# Arrays for evaluation
tr_eval, te_eval = [], []

# Training model
for iter in tqdm.tqdm_notebook(range(epochs)):
    for i in range(tr_length):
        j = np.random.randint(0, tr_length)
        gl = gradient_loss(x_train, y_train, w, j)
        w = w - eta * gl

    tr_pred = [model(x_train[i], w) for i in range(tr_length)]
    te_pred = [model(x_test[i], w) for i in range(te_length)]
    
    tr_eval.append(evaluation(tr_pred, x_train, y_train, w))
    te_eval.append(evaluation(te_pred, x_test, y_test, w))

# Plotting loss and accuracy 
figure(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(1, 2, 1)
plt.plot([l[0] for l in tr_eval], label='train')
plt.plot([l[0] for l in te_eval], label='test')
plt.xlabel('Loss')

plt.subplot(1, 2, 2)
plt.plot([a[1] for a in tr_eval], label='train')
plt.plot([a[1] for a in te_eval], label='test')
plt.xlabel('Accuracy')
plt.legend()


In [ ]:
test_data = test['data']
test_data = data_normalization(test_data)

Y_pred = [model(w, test_data[i]) for i in range(test_data.shape[0])]

# Saving data
with open('solution.csv', 'w') as fout:
    print("Id,Prediction", file=fout)
    for i in range(len(test_data)):
        print(i, Y_pred[i], sep=',', file=fout)